# Project 4


**Option 3: Numerical simulation of heat conduction**

Heat conduction can be modeled by the so-called **heat equation** (aka the diffusion equation).  Using numerical methods described in the following paper, simulate the heat equation and thus reproduce figures 2 and 3 in the paper:

[Numerical Solution to Transient Heat Flow Problems](https://aapt.scitation.org/doi/abs/10.1119/1.1987281), Kobiske, Hock

You are encouraged to explore more heat conduction problems numerically using these techniques if you wish.  You may use any other resources you wish as long as you cite them.

## Abstract

## Physical System and Model

## Algorithms

In [1]:
from proj_4 import *

## Code and Analysis

## Results

## References